<a href="https://colab.research.google.com/github/james-monahan/Code-school-notebooks/blob/main/Week-17-API-GeoCoding/GeoCoding_Nominatim_OSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
The Nominatim API is a REST API (provided by OpenStreetMap), that returns geographic coordinates (latitude and longitude) from a postal address. If several postal addresses (due to inaccuracy or non-existent street number) corresponding to the request are found, the API returns several coordinates, each time with an `importance` score. The coordinates are given in descending order of importance, so you can select only the first address.

https://nominatim.org/release-docs/develop/api/Search/

In [ ]:
import unicodedata
import string

In [ ]:
# Execute the code below :

import requests
link = "https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni,Catania,Italy&format=json"
r = requests.get(link).json()
r

[{'boundingbox': ['37.5110212', '37.5113841', '15.0942789', '15.0969174'],
  'class': 'highway',
  'display_name': 'Via Pietro Mascagni, Picanello-Ognina-Barriera-Canalicchio, Catania, Sicilia, 95129, Italia',
  'importance': 0.51,
  'lat': '37.5112428',
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'lon': '15.0957972',
  'osm_id': 409895559,
  'osm_type': 'way',
  'place_id': 185486160,
  'type': 'residential'},
 {'boundingbox': ['37.5115262', '37.5119158', '15.0979777', '15.1011722'],
  'class': 'highway',
  'display_name': 'Via Pietro Mascagni, Picanello-Ognina-Barriera-Canalicchio, Catania, Sicilia, 95127, Italia',
  'importance': 0.51,
  'lat': '37.5117512',
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'lon': '15.0995581',
  'osm_id': 31183303,
  'osm_type': 'way',
  'place_id': 98616498,
  'type': 'residential'},
 {'boundingbox': ['37.6088571', '37.6091358', '15.1591549', '15.1602847'],
  'clas

In [ ]:
# Here we select only the first address (index 0)
print("First address :", r[0])
print("First address longitude :",r[0]['lon'])
print("First address latitude :",r[0]['lat'])

First address : {'place_id': 185486160, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 409895559, 'boundingbox': ['37.5110212', '37.5113841', '15.0942789', '15.0969174'], 'lat': '37.5112428', 'lon': '15.0957972', 'display_name': 'Via Pietro Mascagni, Picanello-Ognina-Barriera-Canalicchio, Catania, Sicilia, 95129, Italia', 'class': 'highway', 'type': 'residential', 'importance': 0.51}
First address longitude : 15.0957972
First address latitude : 37.5112428


In [ ]:
# For ease of use, you can add a limit on the number of items returned:

link = "https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni,Catania,Italy&format=json"
r = requests.get(link).json()
print("WITHOUT limit, how many coordinates does this address return?",len(r))

link = "https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni,Catania,Italy&format=json&limit=1"
r = requests.get(link).json()
print("WITH limit, how many coordinates does this address return?",len(r))

WITHOUT limit, how many coordinates does this address return? 10
WITH limit, how many coordinates does this address return? 1


## How to create your API query
It's up to you to modify the string to create the right request URL

In [ ]:
# We observe that the query consists of a fixed part, followed by the address to be searched for.
# An URL cannot contain a " " space character, 
# and special characters or accents should be avoided if possible

link_main = 'https://nominatim.openstreetmap.org/?q='
address = '54 Via Pietro Mascagni, Catania, Italy'
link_end = '&format=json&limit=1'

link = link_main + address.replace(" ", '').replace(",", '')

print(link + link_end)

https://nominatim.openstreetmap.org/?q=54ViaPietroMascagniCataniaItaly&format=json&limit=1


In [ ]:
# Create a function here that turns a postal address into a request URL for the Nominatim API, 
# then makes the request and returns the coordinates :

def API_adresse(adresse_postale):
  link_main = 'https://nominatim.openstreetmap.org/?q='
  no_accents = ''.join((c for c in unicodedata.normalize('NFD', adresse_postale) if unicodedata.category(c) != 'Mn'))
  adresse = ''.join([item for item in no_accents if item not in string.punctuation]).replace(' ', '+')
  link_end = '&format=json&limit=1'
  return link_main + adresse + link_end

In [ ]:
# Test it here:
API_adresse(address)

'https://nominatim.openstreetmap.org/?q=54+Via+Pietro+Mascagni+Catania+Italy&format=json&limit=1'

# DataViz
Latitude & Longitude can be used on visualization tools, whether they are BI tools (PowerBI, Table), or Python DataViz libraries such as Plotly or Folium.

Here we will display a map with Folium.


In [ ]:
# The syntax of Folium is very simple, you start by creating a map centered on a point.
# You can change the default zoom level with the argument "zoom_start".

import folium

# We define a location as a list with 2 values : latitude and longitude.
point = [float(r[0]['lat']), float(r[0]['lon'])]

# We center the map on the location 
m = folium.Map(location=point,zoom_start=7)

# We display the map
m

In [ ]:
# Then you can add landmarks and put a clickable comment
m = folium.Map(location=point, )
folium.Marker(
    location=point,
    popup='a good restaurant'
    ).add_to(m)
m

# Challenge
Here is a DataFrame with restaurants in Catania, Sicily, and their respective addresses. Here is your mission:
- Create a new column "coordinates", which will store the coordinates corresponding to each address (you can use the function you created previously).
- Display a map with the 4 restaurant markers. Be careful, the restaurants are very close, remember to set the default zoom level so that it is clearly legible. You can center the map on the first restaurant. And display the name of the restaurant in the tooltip popup.

In [ ]:
import pandas as pd
restaurants = pd.DataFrame([["Gelateria Zio Pietro dal 1964", "Via Porta di Ferro, 47, 95131 Catania CT, Italy"],
                            ["A Casa d'Amici","Via Fischetti, 14, 95131 Catania CT, Italy"],
                            ["La Bitta", "di, Via Acquicella Porto, 95121 Catania CT, Italy"],
                            ["Steak House", "Via Porta di Ferro, 8, 95100 Catania CT, Italy"]
                            ],
                           columns = ["name", "address"])
                           
restaurants

,name,address
0,Gelateria Zio Pietro dal 1964,"Via Porta di Ferro, 47, 95131 Catania CT, Italy"
1,A Casa d'Amici,"Via Fischetti, 14, 95131 Catania CT, Italy"
2,La Bitta,"di, Via Acquicella Porto, 95121 Catania CT, Italy"
3,Steak House,"Via Porta di Ferro, 8, 95100 Catania CT, Italy"


In [ ]:
def get_coord(adresse):
  link = API_adresse(adresse)
  r = requests.get(link).json()
  lat = float(r[0]['lat'])
  lon = float(r[0]['lon'])
  return [lat, lon]

restaurants['coord'] = restaurants['address'].apply(get_coord)

In [ ]:
m = folium.Map(location=restaurants['coord'][0],zoom_start=14 )


folium.Marker(
    location=restaurants['coord'][0],
    popup=restaurants['name'][0],
    icon=folium.Icon(color="red", icon="info-sign"),
).add_to(m)

folium.Marker(
    location=restaurants['coord'][1],
    popup=restaurants['name'][1],
    icon=folium.Icon(color="blue", icon="info-sign"),
).add_to(m)

folium.Marker(
    location=restaurants['coord'][2],
    popup=restaurants['name'][2],
    icon=folium.Icon(color="green", icon="info-sign"),
).add_to(m)

folium.Marker(
    location=restaurants['coord'][3],
    popup=restaurants['name'][3],
    icon=folium.Icon(color="black", icon="info-sign"),
).add_to(m)

m

# Remarks on the Nominatim API
As indicated in the quest, there are many resources for geocoding. Most of them are available by registration, and some require a fee.

The Nominatim API is free and without registration. The disadvantage is that it is relatively slow. If you need to use it in the future, remember to store the results so you don't have to run it several times.

For your knowledge, there is also :
- the **reverse** address API, which allows you to find the nearest postal address using geographic coordinates.
- the API from a **CSV file** if you have a lot of lines to geocode
- the API **GeoJSON** which allows to obtain a geoJSON format of locations to make choropleth maps


All the [documentation is available here](https://nominatim.org/release-docs/develop/api/Search/).